<a href="https://colab.research.google.com/github/DBPReading/CSMBD21_Coursework1_29842451/blob/main/Cloudcomputingcourseworkmultithread.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing modules

In [1]:
#visualisation and processing modules
import numpy as np
import pandas as pd
import requests
from sklearn.model_selection import KFold
import datetime

#threading modules
import logging
import threading
import time
from threading import Thread
from time import sleep, perf_counter

Reading in of Datasets

In [2]:
#Prepare column names
columns1=["Passenger_ID", "Flight_ID", "From_Airport_Code", "To_Airport_Code","Departure_Time", "Duration"]
columns2=["Airport","From_Airport_Code", "Latitude","Longitude"]
#Read in both data sets
airport=pd.read_csv('AComp_Passenger_data_no_error(1).csv',names=columns1)
latlong=pd.read_csv('Top30_airports_LatLong(1).csv',names=columns2)



In [ ]:
#Basic overview of airport dataset
print(airport) 

    Passenger_ID Flight_ID From_Airport_Code To_Airport_Code  Departure_Time  \
0     UES9151GS5  SQU6245R               DEN             FRA      1420564460   
1     UES9151GS5  XXQ4064B               JFK             FRA      1420563917   
2     EZC9678QI6  SOH3431A               ORD             MIA      1420563649   
3     ONL0812DH1  SOH3431A               ORD             MIA      1420563649   
4     CYJ0225CH1  PME8178S               DEN             PEK      1420564409   
..           ...       ...               ...             ...             ...   
495   BWI0520BG6  BER7172M               KUL             LAS      1420565167   
496   LLZ3798PE3  EWH6301Y               CAN             DFW      1420564967   
497   KKP5277HZ7  KJR6646J               IAH             BKK      1420565203   
498   JJM4724RF7  XXQ4064B               JFK             FRA      1420563917   
499   SJD8775RZ4  WSK1289Z               CLT             DEN      1420563542   

     Duration  
0        1049  
1      

In [4]:
#Basic overview of latitude and logitude dataset
print(latlong)

              Airport From_Airport_Code   Latitude   Longitude
0             ATLANTA               ATL  33.636719  -84.428067
1             BEIJING               PEK  40.080111  116.584556
2              LONDON               LHR  51.477500   -0.461389
3             CHICAGO               ORD  41.978603  -87.904842
4               TOKYO               HND  35.552258  139.779694
5         LOS ANGELES               LAX  33.942536 -118.408075
6               PARIS               CDG  49.012779    2.550000
7   DALLAS/FORT WORTH               DFW  32.896828  -97.037997
8           FRANKFURT               FRA  50.026421    8.543125
9           HONG KONG               HKG  22.308919  113.914603
10             DENVER               DEN  39.861656 -104.673178
11              DUBAI               DXB  25.252778   55.364444
12            JAKARTA               CGK  -6.125567  106.655897
13          AMSTERDAM               AMS  52.308613    4.763889
14             MADRID               MAD  40.493556   -3

Task 1:

In [3]:
###Restrict Airport data set to airport codes and index
airport1=airport.loc[:,'From_Airport_Code']

###Restrict Latlong data set to airport names and corresponding airport codes
latlong1=latlong[['Airport','From_Airport_Code']].set_index('Airport')

###Restrict Airport data set to passenger id and index
passenger1=airport.loc[:,'Passenger_ID']
#
#print(airport1)
#print(latlong1)
#print(passenger1)

In [4]:
#Task 1: Determine the total number of flights from each airport

#Initialise lists
targetlist = []
dictionarylist = []

#Function for creating a list of indices for the partitioned sets
def divider(entryfile,split): 
  filevalue, splitvalue = divmod(entryfile.size, split)
  return (entryfile[i*filevalue+min(i, splitvalue):(i+1)*filevalue+min(i+1, splitvalue)] for i in range(split))

#Function for partitioning the sets in preperation for threading
def partitioner1(entryfile,split,i):
  if entryfile.size % split == 0:
      divider(entryfile,split)
      listing= list(divider(airport1,split))
      
      index=listing[i]
      
      return index
  else:
    print(str(entryfile.size) + ' cannot be subdivided by '+ str(split))
    raise ValueError


def read_input(entryfile):
  for line in entryfile:
  # Split the line into words
    yield line.split()

def inputmerge(mergefile,mergecode):
  #Input Segment
  file_dict = mergefile.to_dict(orient="dict")[mergecode]
  inv_file_dict = {value: key for key, value in file_dict.items()}
  #Output: <Airport, FAA Code> -> <FAA Code, Airport>
  return inv_file_dict

def threadsection1(mergefile,countfile,inversedict,split):
  start_time = perf_counter()
  threads=[]
  for i in range(split):
    print("Activate thread "+ str(i+1))
    parting=partitioner1(countfile,split,i )
    thread = Thread(target=map1(mergefile,parting,inversedict))
    # start the threads
    thread.start()
    threads.append(thread)
    # wait for the threads to complete
  for i in range(len(threads)):
    thread=threads[i]
    thread.join()
    print("End thread "+ str(i+1))
  end_time = perf_counter()
  print(f'It took {end_time- start_time: 0.2f} second(s) to complete.')



def map1(mergefile,countfile,passing):
  #Map Segment
  #Output: <FAA Code, x> -> <Airport, x>  where x is count
  data = read_input(countfile)
  for words in data:
    for word in words:
      #check to ensure mergefile is present, otherwise ignore
      try:
        targetcode= passing[word] 
      except NameError:
        targetcode= word 
        pass 
        
      dictkey= {targetcode: 1}
      dictionarylist.append(dict(dictkey))

#Reduce segment
def reducer1():
  #sorter segment, sorts dictionaries alphanumerically pre reduction
  dictionary= sorted(dictionarylist, key= lambda d : list(d.keys()))

  for key, value in dictionary[0].items():
    current_target,current_count = [str(key),0] 

  for diction in dictionary:
    for key, value in diction.items():
      if key ==current_target:
        current_count= current_count+value
      else:
        targetkey= {current_target: current_count}
        targetlist.append(dict(targetkey))
        current_target = key
        current_count = value


def main(mergefile,countfile,mergecode,split):
  #map + reduce segments
  try:
    inversedict=inputmerge(mergefile,mergecode)
  except NameError:
    pass
  threadsection1(mergefile,countfile,inversedict,split)
  reducer1()

if __name__ == "__main__":
  main(latlong1,airport1,'From_Airport_Code',4)
print(dictionarylist)

#print(dictionary)
print(targetlist)

Activate thread 1
Activate thread 2
Activate thread 3
Activate thread 4
End thread 1
End thread 2
End thread 3
End thread 4
It took  0.01 second(s) to complete.
[{'DENVER': 1}, {'NEW YORK': 1}, {'CHICAGO': 1}, {'CHICAGO': 1}, {'DENVER': 1}, {'KUALA LUMPUR': 1}, {'MADRID': 1}, {'DENVER': 1}, {'LONDON': 1}, {'NEW YORK': 1}, {'JAKARTA': 1}, {'MUNICH': 1}, {'AMSTERDAM': 1}, {'DALLAS/FORT WORTH': 1}, {'NEW YORK': 1}, {'DENVER': 1}, {'NEW YORK': 1}, {'MUNICH': 1}, {'KUALA LUMPUR': 1}, {'MIAMI': 1}, {'PARIS': 1}, {'GUANGZHOU': 1}, {'LONDON': 1}, {'CHICAGO': 1}, {'DENVER': 1}, {'HOUSTON': 1}, {'MUNICH': 1}, {'GUANGZHOU': 1}, {'CHICAGO': 1}, {'MIAMI': 1}, {'LAS VEGAS': 1}, {'PARIS': 1}, {'CHICAGO': 1}, {'CHARLOTTE': 1}, {'JAKARTA': 1}, {'LAS VEGAS': 1}, {'ATLANTA': 1}, {'DENVER': 1}, {'PARIS': 1}, {'LONDON': 1}, {'CHICAGO': 1}, {'KUALA LUMPUR': 1}, {'CHARLOTTE': 1}, {'CHICAGO': 1}, {'SHANGHAI': 1}, {'DENVER': 1}, {'JAKARTA': 1}, {'LONDON': 1}, {'SHANGHAI': 1}, {'ROME': 1}, {'KUALA LUMPUR': 1}, 

In [7]:
###Task 2: Determine the passenger having had the highest number of flights.

targetlist = []
dictionarylist2 = []
#Output: <Passenger code, x> where x is count, then work out max value

def map2(inputfile):
  #Map Segment
  inputdata = read_input(inputfile)
  for words in inputdata:
    dictkey2= {word: 1 for word in words}
    dictionarylist2.append(dict(dictkey2))
  
#Reduce segment
def reducer2():
  #sorter segment, sorts dictionaries alphanumerically pre reduction
  dictionary2= sorted(dictionarylist2, key= lambda d : list(d.keys()))

  #reduce segment
  for key, value in dictionary2[0].items():
    current_passenger,current_count = [str(key),0] 
  for diction2 in dictionary2:
    for key, value in diction2.items():
      if key ==current_passenger:
        current_count= current_count+value
      else:
        targetkey= {current_passenger: current_count}
        targetlist.append(dict(targetkey))
        current_passenger = key
        current_count = value

def main(inputfile):
  #map + reduce segments
  map2(inputfile)
  reducer2()
  #mosttarget= sorted(targetlist, key= lambda value: int(value)) )
  mosttarget = max(targetlist, key=lambda d:list(d.values()))
  print(mosttarget)


if __name__ == "__main__":
  main(passenger1)

print(targetlist)




{'UES9151GS5': 25}
[{'BWI0520BG6': 23}, {'CDC0302NN5': 12}, {'CKZ3132BR4': 19}, {'CXN7304ER2': 17}, {'CYJ0225CH1': 11}, {'DAZ3029XA0': 23}, {'EDV2089LK5': 13}, {'EZC9678QI6': 21}, {'HCA3158QA6': 21}, {'HGO4350KK1': 18}, {'IEG9308EA5': 10}, {'JBE2302VO4': 16}, {'JJM4724RF7': 21}, {'KKP5277HZ7': 11}, {'LLZ3798PE3': 16}, {'MXU9187YC7': 14}, {'ONL0812DH1': 12}, {'PAJ3974RK1': 10}, {'PIT2755XC1': 8}, {'POP2875LH3': 19}, {'PUD8209OG3': 23}, {'SJD8775RZ4': 16}, {'SPR4484HA6': 23}, {'UES9151GS5': 25}, {'UMH6360YP0': 1}, {'VZY2993ME1': 16}, {'WBE6935NU3': 19}, {'WTC9125IE5': 14}, {'WYU2010YH8': 19}, {'XFG5747ZT9': 13}]
